# 飞桨常规赛：遥感影像地块分割 - 4月第10名方案

## 1. 准备下载相关库与数据解压

In [ ]:
#下载PaddleSeg
!git clone https://gitee.com/paddlepaddle/PaddleSeg.git

Cloning into 'PaddleSeg'...
remote: Enumerating objects: 10924, done.
remote: Counting objects: 100% (10924/10924), done.
remote: Compressing objects: 100% (5412/5412), done.
remote: Total 10924 (delta 7389), reused 8171 (delta 5353), pack-reused 0
Receiving objects: 100% (10924/10924), 156.94 MiB | 20.03 MiB/s, done.
Resolving deltas: 100% (7389/7389), done.
Checking connectivity... done.


In [ ]:
!unzip -q data/data77571/train_and_label.zip > /dev/null
!unzip -q data/data77571/img_test.zip > /dev/null

In [ ]:
!rm -r __MACOSX/

## 2. 数据标签分析

In [ ]:
#测试输出图片
pic1 = '/home/aistudio/lab_train/T020005.png'
from PIL import Image
import numpy as np
img = Image.open(pic1)
test  = np.array(img)
# print(test.shape)
a = test.flatten()
set(a.tolist())

{1, 255}

### 2.1 数据类型分布的分析
<font size="5pt" color="red">注意：下面这部分数据分析的代码参考[坑姐](https://aistudio.baidu.com/aistudio/personalcenter/thirdview/90149)的[项目](https://aistudio.baidu.com/aistudio/projectdetail/1752986)，这部分数据分析的代码可以用于任何类似的任务，非常有学习的价值！</font>

In [ ]:
import cv2, os
import numpy as np

#amount of classer
CLASSES_NUM = 4

#find imagee in folder dir
def findImages(dir,topdown=True):
    im_list = []
    if not os.path.exists(dir):
        print("Path for {} not exist!".format(dir))
        raise
    else:
        for root, dirs, files in os.walk(dir, topdown):
            for fl in files:
                im_list.append(fl)
    return im_list

# amount of images corresponding to each classes
images_count = [0]*CLASSES_NUM
# amount of pixels corresponding to each class
class_pixels_count = [0]*CLASSES_NUM
# amount of pixels corresponding to the images of each class
image_pixels_count = [0]*CLASSES_NUM

image_folder = './lab_train/'
im_list = findImages(image_folder) 

In [ ]:
#######################################################注意！！！！！！！！！！！
# 这部分代码没事就不要再次运行了，非常消耗时间！！
for im in im_list:
    # print(im)
    cv_img = cv2.imread(os.path.join(image_folder, im), cv2.IMREAD_UNCHANGED)
    size_img = cv_img.shape
    colors = set([])
    for i in range(size_img[0]):
        for j in range(size_img[1]):
            p_value = cv_img.item(i,j)
            if not p_value < CLASSES_NUM: # check
                pass
                # print(p_value) #因为有255，所以不打印了
            else:
                class_pixels_count[p_value] = class_pixels_count[p_value] + 1
                colors.add(p_value)
    im_size = size_img[0]*size_img[1]
    for n in range(CLASSES_NUM):
        if n in colors:
            images_count[n] = images_count[n] + 1
            image_pixels_count[n] = image_pixels_count[n] + im_size

print(images_count)
print(class_pixels_count)
print(image_pixels_count)

[23707, 30151, 23044, 18978]
[247142772, 310655521, 152480301, 159504650]
[1553661952, 1975975936, 1510211584, 1243742208]


In [ ]:
# t = class_pixels_count.copy()
t = [247142772, 310655521, 152480301, 159504650]
a = np.array(t).sum()
for i in range(len(t)):
    t[i] = round(t[i]/a, 4)
print(t)

[0.2841, 0.3572, 0.1753, 0.1834]


<font color="red" size="5pt">小结：根据打印结果可知，可以看出类别比较均衡</font>

### 2.2 数据准备（生成用于训练得到标签文件）

In [ ]:
%cd ~

/home/aistudio


In [ ]:
import os
from imghdr import what
import numpy as np
import random

def get_all_date(dir_images, dir_masks):
    """生成训练、测试所需的txt文件"""
    res = []
    img_list = os.listdir(dir_images)
    mask_list = os.listdir(dir_masks)
    assert len(img_list) == len(mask_list)  #样本和标签数量肯定要一样
    img_suffix = os.path.splitext(os.path.join(dir_images,img_list[0]))[1]
    mask_suffix = os.path.splitext(os.path.join(dir_masks,mask_list[0]))[1]
    print(f'The suffix of images is: {img_suffix} \n The suffix of masks is {mask_suffix}!')
    for i in range(len(img_list)):
        img_name = img_list[i]
        mask_name = os.path.splitext(img_name)[0]+mask_suffix
        image_path = os.path.join(dir_images,img_name)
        mask_path = os.path.join(dir_masks,mask_name)
        if os.path.splitext(img_name)[0] == os.path.splitext(mask_name)[0] and what(image_path) and what(mask_path): 
            #样本和标签文件名一样（不包含后缀，因为后缀可能不一样），同时都为图片
            res.append((image_path, mask_path))
        else:
            print(image_path, mask_path)
    return res

res = get_all_date('img_train', 'lab_train')
print(f'The samples of image and label is {res[:3]}')
print(f'The effective number of image data is: {len(res)}')
random.shuffle(res)
random.shuffle(res)
pro = 0.95 #训练/验证比例
pro_int = int(len(res)*pro)
print(f'The split number of train data is: {pro_int}')
with open('./train_list.txt', 'w') as f:
    for line in res[:pro_int]:
        f.writelines(line[0] + ' ' + line[1] + '\n')

with open('./val_list.txt', 'w') as f:
    for line in res[pro_int:]:
        f.writelines(line[0] + ' ' + line[1] + '\n')

The suffix of images is: .jpg 
 The suffix of masks is .png!
The samples of image and label is [('img_train/T057389.jpg', 'lab_train/T057389.png'), ('img_train/T040500.jpg', 'lab_train/T040500.png'), ('img_train/T039138.jpg', 'lab_train/T039138.png')]
The effective number of image data is: 66652
The split number of train data is: 63319


## 3. 配置训练文件

In [4]:
#进入PaddleSeg目录
%cd ./PaddleSeg/

/home/aistudio/PaddleSeg


<font size="4">根据[自定义数据集文档](https://gitee.com/paddlepaddle/PaddleSeg/blob/release/v2.0/docs/data_prepare.md)自己继承并实现配置文件`PaddleSeg/configs/u2net/My_u2net_cityscapes_1024x512.yml`，运行下面的命令可以查看该文件的内容</font>

In [ ]:
!cat /home/aistudio/PaddleSeg/configs/u2net/My_u2net_cityscapes_1024x512.yml































  coef: [1, 1, 1, 1, 1, 1, 1]

### 3.1 第一次训练

In [ ]:
#训练——注意：evaluate会在每次保存save_interval的时候进行一次！
!python train.py \
       --config configs/deeplabv3/My_deep.yml \
       --do_eval \
       --use_vdl \
       --save_interval 250 \
       --save_dir output_deeplabv3

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/setuptools/depends.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
2021-04-12 21:32:38 [INFO]	
------------Environment Information-------------
platform: Linux-4.4.0-150-generic-x86_64-with-debian-stretch-sid
Python: 3.7.4 (default, Aug 13 2019, 20:35:49) [GCC 7.3.0]
Paddle compiled with cuda: True
NVCC: Cuda compilation tools, release 10.1, V10.1.243
cudnn: 7.6
GPUs used: 1
CUDA_VISIBLE_DEVICES: None
GPU: ['GPU 0: Tesla V100-SXM2-32GB']
GCC: gcc (Ubuntu 7.5.0-3ubuntu1~16.04) 7.5.0
PaddlePaddle: 2.0.1
OpenCV: 4.1.1
------------------------------------------------
2021-04-12 21:32:39 [INFO]	
---------------Config Information---------------
batch_size: 16
iters: 10000
learning_rate:
  decay:
    end_lr: 0
    power: 0.9
    type: poly
  value: 0.01
loss:
  coef:
  - 1
  types:
  - ignore_index: 255
    type: CrossEntropyLoss
mo

### 3.2 从第28500次迭代处进行第二次训练

In [31]:
# 也可以在项目提供的7500轮训练结果上继续训练
!python train.py \
       --config configs/deeplabv3/My_deep.yml \
       --resume_model output_deeplabv3/iter_28500 \
       --do_eval \
       --use_vdl \
       --save_interval 750 \
       --save_dir output_deeplabv3

## 3. 进行预测
将最佳结果的模型参数`output_deeplabv3/best_model/`文件夹拷贝到个人目录`~`

PS(postscript，补充一下)：PaddleSeg比较贴心，默认训练的时候会保存最佳的结果，希望以后官方的其他库也能实现这个功能

In [5]:
!python predict.py \
       --config configs/deeplabv3/My_deep.yml \
       --model_path ~/best_model/model.pdparams \
       --image_path ../img_testA/ \
       --save_dir ../submit/

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):
2021-05-10 07:31:59 [INFO]	
---------------Config Information---------------
batch_size: 16
iters: 50000
learning_rate:
  decay:
    end_lr: 0
    power: 0.9
    type: poly
  value: 0.001
loss:
  coef:
  - 1
  types:
  - type: CrossEntropyLoss
model:
  align_corners: false
  aspp_out_channels: 256
  aspp_ratios:
  - 1
  - 12
  - 24
  - 36
  backbone:
  

<font size="4pt">最后将`~/submit/`下面的`pseudo_color_prediction`文件夹重命名为`resutl`，该文件夹就是最终的结果文件夹，里面是分割后的预测图片</font>

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 